# Homework IV - ME 364 (Spring 2022)

### <font color='red'>__Question I__</font> 
A series of measurements have been done on a vibrating system, which we can be simplified as a single degree-of-freedom. The stiffness of the system was changed and the time period of the system was measured. The data is recorded in a dataset and attached to this assingment with the name `Q1.csv`. This dataset includes stiffness values and the corresponding time period for the oscillation. Use <u>simple linear regression</u> to estimate the system's mass.

<font color='blue'>__NOTES :__</font>
- For a system with single degree-of-freedom, $\omega_n=\sqrt{\frac{k}{m}}$
- The time period of a system and its natural frequency are related with $\omega_n=\frac{2\pi}{T}$



In [ ]:
# Your code goes here


### <font color='red'>__Question II__</font>


We are trying to model a system with two degrees of freedom (https://www.brown.edu/Departments/Engineering/Courses/En4/Notes/vibrations_mdof/vibrations_mdof.htm) based on some measurement data. We have the magnitude of input force and the corresponding maximum displacements of mass 1 and mass 2 for given input forces. The data is stored in the csv file `Q2.csv`. 

- Use <u>Linear Regression</u> to develop a model to predict the magnitude of the input force if we know the displacements of mass 1 and mass 2.  

- Use evaluation metrics to assess the performance of your model. Is this a good model? Why? Run your model multiple times and look at the evaluation metrics. Do you see anything unusual about these metrics?

- What is the predicted force magnitude when mass 1 and mass 2 displacements both are 0.12 m ?

In [ ]:
# Your code goes here
